![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# **MedicalNerApproach**

In this notebook, we will examine the `MedicalNerApproach` to to train an `MedicalNerModel`.


**📖 Learning Objectives:**

1. Understand the meaning and usage of Named Entity Recognition.

2. Learn how to preprocess your data before training a model.

3. Understand how to train a Named Entity Recognition model using `MedicalNerApproach`.




**🔗 Helpful Links:**

For extended examples of usage, see the [Spark NLP Workshop](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/01.0.Clinical_Named_Entity_Recognition_Model.ipynb)

Python Documentation: [MedicalNerApproach](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/ner/medical_ner/index.html#sparknlp_jsl.annotator.ner.medical_ner.MedicalNerApproach.po)

Scala Documentation: [MedicalNerApproach](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/ner/MedicalNerApproach.html) <br/>


**Blogposts and videos:**


- [Named Entity Recognition (NER) with BERT in Spark NLP](https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77)

- [State of the art Clinical Named Entity Recognition in Spark NLP - Youtube](https://www.youtube.com/watch?v=YM-e4eOiQ34)

- [Named Entity Recognition for Healthcare with SparkNLP NerDL and NerCRF](https://medium.com/spark-nlp/named-entity-recognition-for-healthcare-with-sparknlp-nerdl-and-nercrf-a7751b6ad571)

- [Named Entity Recognition for Clinical Text](https://medium.com/atlas-research/ner-for-clinical-text-7c73caddd180)

## **📜 Background**


`MedicalNerApproach` is used for training generic NER models based on Neural Networks.

The architecture of the neural network is a Char CNNs - BiLSTM - CRF that achieves state-of-the-art in most datasets.

In the original framework, the CNN extracts a fixed length feature vector from character-level features. For each word, these vectors are concatenated and fed to the BLSTM network and then to the output layers. They employed a stacked bi-directional recurrent neural network with long short-term memory units to transform word features into named entity tag scores. The extracted features of each word are fed into a forward LSTM network and a backward LSTM network. The output of each network at each time step is decoded by a linear layer and a log-softmax layer into log-probabilities for each tag category. These two vectors are then simply added together to produce the final output. In the architecture of the proposed framework in the original paper, 50-dimensional pretrained word embeddings is used for word features, 25-dimension character embeddings is used for char features, and capitalization features (allCaps, upperInitial, lowercase, mixedCaps, noinfo) are used for case features. <br/>








## **🎬 Colab Setup**

In [ ]:
!pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [4]:
from johnsnowlabs import nlp, medical

spark = nlp.start()

👌 Detected license file /content/4.4.3.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.4.1, 💊Spark-Healthcare==4.4.3, running on ⚡ PySpark==3.1.2


In [9]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

## **🖨️ Input/Output Annotation Types**

- Input: `DOCUMENT`, `TOKEN`, `WORD_EMBEDDINGS`

- Output: `NAMED_ENTITY`

## **📂 Training Data**

The training data should be a labeled Spark Dataset, in the format of CoNLL 2003 IOB with Annotation type columns. The data should have columns of type `DOCUMENT`, `TOKEN`, `WORD_EMBEDDINGS` and an additional label column of annotator type `NAMED_ENTITY`.

Excluding the label, this can be done with for example:

- SentenceDetector
- Tokenizer
- WordEmbeddingsModel (any embeddings can be chosen, e.g. BertEmbeddings for BERT based embeddings). <br/>



`CoNLL(includeDocId=True).readDataset(spark, "conll_file.txt")`  this methos can allow if you have doc_id information in the conll file, you can add this information to the dataframe as a column. 

```
conll="""-DOCSTART- -X- -1- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O

-DOCSTART- -X- 2 O

Rare NNP B-NP O
Hendrix NNP I-NP B-PER

-DOCSTART- -X- -3-1- O

China NNP B-NP B-LOC
says VBZ B-VP O

-DOCSTART-

China NNP B-NP B-LOC
says VBZ B-VP O
"""
```
```
with open('conll_file.txt', 'w') as f:
    f.write(conll)

data = CoNLL(includeDocId=True).readDataset(spark, "conll_file.txt")

data.show()
```



```
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|doc_id|                text|            document|            sentence|               token|                 pos|               label|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     1|EU rejects German...|[{document, 0, 28...|[{document, 0, 47...|[{token, 0, 1, EU...|[{pos, 0, 1, NNP,...|[{named_entity, 0...|
|     2|Rare Hendrix song...|[{document, 0, 97...|[{document, 0, 50...|[{token, 0, 3, Ra...|[{pos, 0, 3, NNP,...|[{named_entity, 0...|
|   3-1|China says Taiwan...|[{document, 0, 13...|[{document, 0, 46...|[{token, 0, 4, Ch...|[{pos, 0, 4, NNP,...|[{named_entity, 0...|
|     X|China says Taiwan...|[{document, 0, 13...|[{document, 0, 46...|[{token, 0, 4, Ch...|[{pos, 0, 4, NNP,...|[{named_entity, 0...|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
```



We will download NCBI Disease Dataset for the training.

In [5]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltrain.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltest.txt

In [6]:
conll_data = nlp.CoNLL().readDataset(spark, 'NER_NCBIconlltrain.txt')

conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
|The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [7]:
conll_data.count()

3266

In [10]:
conll_data.select(F.explode(F.arrays_zip(conll_data.token.result,
                                         conll_data.label.result)).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth"))\
          .groupBy('ground_truth')\
          .count()\
          .orderBy('count', ascending=False)\
          .show(100,truncate=False)

+------------+-----+
|ground_truth|count|
+------------+-----+
|O           |75093|
|I-Disease   |3547 |
|B-Disease   |3093 |
+------------+-----+



In [11]:
conll_data.select("label.result").distinct().count()

1537

As you can see, there are too many `O` labels in the dataset. 
To make it more balanced, we can drop the sentences have only O labels.
(`c>1` means we drop all the sentences that have no valuable  labels other than `O`) 

```
conll_data = conll_data.withColumn('unique', F.array_distinct("label.result"))\
                       .withColumn('c', F.size('unique'))\
                       .filter(F.col('c')>1)

conll_data.select(F.explode(F.arrays_zip(conll_data.token.result,conll_data.label.result)).alias("cols")) \
          .select(F.expr("cols['0']").alias("token"),
                  F.expr("cols['1']").alias("ground_truth"))\
          .groupBy('ground_truth')\
          .count()\
          .orderBy('count', ascending=False)\
          .show(100,truncate=False)
```


In [12]:
# Clinical word embeddings trained on PubMED dataset
clinical_embeddings = nlp.WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


Preparing the test data for the model evaluation

In [13]:
test_data = nlp.CoNLL().readDataset(spark, 'NER_NCBIconlltest.txt')

test_data = clinical_embeddings.transform(test_data)

test_data.write.mode("overwrite").parquet('NER_NCBIconlltest.parquet')

## **✅ Graph Creation**

We will use `TFGraphBuilder` annotator which can be used to create graphs in the model training pipeline. `TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow is available. The graph is stored in the defined folder and loaded by the `MedicalNerApproach` annotator.

In [ ]:
!pip install -q tensorflow==2.11.0
!pip install -q tensorflow-addons

In [15]:
graph_folder_path = "medical_ner_graphs"

ner_graph_builder = medical.TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(20)\
    .setIsLicensed(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

## **🔎 Parameters**


- `batchSize`: (Int) Size for each batch in the optimization process (Default: 64).

- `dropout`: (Float) Dropout at the output of each layer (Default: 0.05f)

- `enableOutputLogs`: (Bool) Whether to output to annotators log folder (Default: false).

- `maxEpochs`: (Int) Maximum number of epochs to train (Default: 70).

- `minEpochs`: (Int) Maximum number of epochs to train (Default: 0).

- `graphFile`: (Str) File path that contains external graph file.

- `graphFolder`: (Str) Folder path that contain external graph files.

- `includeConfidence`: (Bool) Whether to include confidence scores in annotation metadata. Setting this parameter to True will add the confidence score to the metadata of the NAMED_ENTITY annotation. In addition, if includeAllConfidenceScores is set to True, then the confidence scores of all the tags will be added to the metadata, otherwise only for the predicted tag (the one with maximum score) (Default: False). 

- `includeAllConfidenceScores`: (Bool) Whether to include confidence scores for all tags in annotation metadata or just the score of the predicted tag, by default False.
Needs the includeConfidence parameter to be set to True. Enabling this may slow down the inference speed (Default: False). 

- `labelColumn`: (Str) Column with one label per document.

- `lr`: (Float) Learning rate for the optimization process (Default: 0.001).

- `outputLogsPath`: (Str) Folder path to save training logs. If no path is specified, the logs won’t be stored in disk. The path can be a local file path, a distributed file path (HDFS, DBFS), or a cloud storage (S3).

- `testDataset`: (Str) Path to test dataset in parquet format. If set, the dataset will be used to calculate statistic on it during training.

- `validationSplit`: (Float) Choose the proportion of training dataset to be validated against the model on each Epoch. The value should be between 0.0 and 1.0 and by default it is 0.0 and off. <br/>
The validation dataset is randomly extracted from the training dataset before training starts. If the value is 0.0, then no validation will be performed (hold out data).

- `verbose`: (Int) Level of verbosity during training (Default: 2).

- `randomSeed`: (Int) Random seed. Set to positive integer to get reproducible results (Default: none)

- `evaluationLogExtended`: (Bool) Whether logs for validation to be extended (Default: False)

- `tagsMapping`: (List) A map specifying how old tags are mapped to new ones as a list of comma-separated entities, where the first entity is the old tag and the second entity is the new tag. For example, if the map is set to [“OLDTAG,NEWTAG”, “B-PER,B-VIP”, “I-PER, I-VIP”], then all occurrences of “OLDTAG” will be mapped to “NEWTAG”, all occurrences of “B-PER” will be mapped to “B-VIP”, and all occurrences of “I-PER” will be mapped to “I-VIP”. It only works if overrideExistingTags is set to False.

- `earlyStoppingPatience`: (Int) Number of epochs to wait before early stopping if no improvement, by default 5. Given the earlyStoppingCriterion, if the performance does not improve for the given number of epochs, then the training will stop. If the value is 0, then early stopping will occurs as soon as the criterion is met (no patience).

- `earlyStoppingCriterion` (Float) If set, this param specifies the criterion to stop training if performance is not improving. Default value is 0 which is means that early stopping is not used. <br/>
The criterion is set to F1-score if the validationSplit is greater than 0.0 (F1-socre on validation set) or testDataset is defined (F1-score on test set), otherwise it is set to model loss. The priority is as follows: - If testDataset is defined, then the criterion is set to F1-score on test set. - If validationSplit is greater than 0.0, then the criterion is set to F1-score on validation set. - Otherwise, the criterion is set to model loss. <br/>
Note that while the F1-score ranges from 0.0 to 1.0, the loss ranges from 0.0 to infinity. So, depending on which case you are in, the value you use for the criterion can be very different. For example, if validationSplit is 0.1, then a criterion of 0.01 means that if the F1-score on the validation set difference from last epoch is greater than 0.01, then the training should stop. However, if there is not validation or test set defined, then a criterion of 2.0 means that if the loss difference between the last epoch and the current one is less than 2.0, then training should stop.

- `pretrainedModelPath`: (Str) Path to an already trained MedicalNerModel. <br/>
This pretrained model will be used as a starting point for training the new one. The path can be a local file path, a distributed file path (HDFS, DBFS), or a cloud storage (S3).

- `overrideExistingTags`: Whether to override the already learned tags when using a pretrained model to initialize the new model. A value of True (default) will override existing tags.

- `logPrefix`: (Str) A prefix that will be appended to every log, default value is empty.

- `useBestModel`: (Bool) Whether to restore and use the model from the epoch that has achieved the best performance at the end of the training. By default False (keep the model from the last trained epoch). <br/>
The best model depends on the earlyStoppingCriterion, which can be F1-score on test/validation dataset or the value of loss.


## **🦾 Model Training**

Now, we will define `NerApproach` and generate a pipeline along with clinical embeddings and TFGraph. 

In [18]:
nerTagger = medical.NerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(10)\
    .setBatchSize(64)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')\
    .setTestDataset("NER_NCBIconlltest.parquet")\
    .setGraphFolder(graph_folder_path)
   # .setEnableMemoryOptimizer(True) #>> if you have a limited memory and a large conll file, you can set this True to train batch by batch       

ner_pipeline = nlp.Pipeline(stages=[
          clinical_embeddings,
          ner_graph_builder,
          nerTagger
 ])

Fitting the pipeline with the training data. 

In [19]:
%%time
ner_model = ner_pipeline.fit(conll_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 20}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_20_85.pb
CPU times: user 15.7 s, sys: 194 ms, total: 15.9 s
Wall time: 6min 40s


Checking the results saved in the log file

In [20]:
import os 
log_file= os.listdir("ner_logs")[0]

with open (f"./ner_logs/{log_file}") as f:
  print(f.read())

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_20_85.pb
Training started - total epochs: 10 - lr: 0.001 - batch size: 64 - labels: 3 - chars: 84 - training examples: 3266


Epoch 1/10 started, lr: 0.001, dataset size: 3266


Epoch 1/10 - 44.89s - loss: 424.25015 - avg training loss: 9.866282 - batches: 43
Quality on validation dataset (20.0%), validation examples = 653
time to finish evaluation: 3.29s
Total validation loss: 62.3945	Avg validation loss: 4.7996
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 16	 12	 734	 0.5714286	 0.021333333	 0.041131105
B-Disease	 12	 31	 635	 0.27906978	 0.01854714	 0.034782607
tp: 28 fp: 43 fn: 1369 labels: 2
Macro-average	 prec: 0.4252492, rec: 0.019940237, f1: 0.038094208
Micro-average	 prec: 0.3943662, rec: 0.020042948, f1: 0.038147137
Quality on test dataset: 
time to finish evaluation: 3.56s
Total test loss: 77.3384	Avg test loss: 5.5242
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 18	 13	 771	 0.58064514	 0.022813689	 0.0439024

**Evaluating the model**

In [21]:
pred_df = ner_model.stages[2].transform(test_data)

In [22]:
pred_df.columns

['text', 'document', 'sentence', 'token', 'pos', 'label', 'embeddings', 'ner']

In [23]:
evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+-----+-----+---------+------+------+
| entity|   tp|   fp|   fn|total|precision|recall|    f1|
+-------+-----+-----+-----+-----+---------+------+------+
|Disease|531.0|180.0|173.0|704.0|   0.7468|0.7543|0.7505|
+-------+-----+-----+-----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.7505300353356891|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.7505300353356891|
+------------------+

None


In [24]:
evaler = medical.NerDLMetrics(mode="partial_chunk_per_token")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+------+-----+-----+------+---------+------+------+
| entity|    tp|   fp|   fn| total|precision|recall|    f1|
+-------+------+-----+-----+------+---------+------+------+
|Disease|1210.0|132.0|287.0|1497.0|   0.9016|0.8083|0.8524|
+-------+------+-----+-----+------+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8524128214159915|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8524128214159915|
+------------------+

None


### Train a Model Choosing Best Model

`useBestModel`:  This param preserves and restores the model that has achieved the best performance at the end of the training. The priority is metrics from testDataset (micro F1), metrics from validationSplit (micro F1), and if none is set it will keep track of loss during the training. <br/>

`earlyStopping`: You can stop training at the point when the perforfmance on test/validation dataset starts to degrage. Two params are used in order to use this feature: <br/> 

- `earlyStoppingCriterion`: This is used set the minimal improvement of the test metric to terminate training. The metric monitored is the same as the metrics used in `useBestModel` (micro F1 when using test/validation set, loss otherwise). Default is 0 which means no early stopping is applied. 

- `earlyStoppingPatience`:  The number of epoch without improvement which will be tolerated. Default is 0, which means that early stopping will occur at the first time when performance in the current epoch is no better than in the previous epoch.

In [25]:
# remove the existing logs

! rm -r ./ner_logs

In [26]:
nerTagger = medical.NerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(30)\
    .setBatchSize(64)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')\
    .setTestDataset("NER_NCBIconlltest.parquet")\
    .setUseBestModel(True)\
    .setEarlyStoppingCriterion(0.04)\
    .setEarlyStoppingPatience(3)\
    .setGraphFolder(graph_folder_path)


ner_pipeline = nlp.Pipeline(stages=[
          clinical_embeddings,
          ner_graph_builder,
          nerTagger
 ])

In [27]:
%%time
ner_model = ner_pipeline.fit(conll_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 20}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_20_85.pb
CPU times: user 12 s, sys: 162 ms, total: 12.1 s
Wall time: 4min 26s


Checking the results saved in the log file

In [28]:
import os 
log_file= os.listdir("ner_logs")[0]

with open (f"./ner_logs/{log_file}") as f:
  print(f.read())

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_20_85.pb
Training started - total epochs: 30 - lr: 0.001 - batch size: 64 - labels: 3 - chars: 84 - training examples: 3266


Epoch 1/30 started, lr: 0.001, dataset size: 3266


Epoch 1/30 - 22.27s - loss: 246.34515 - avg training loss: 5.728957 - batches: 43
Quality on validation dataset (20.0%), validation examples = 653
time to finish evaluation: 2.06s
Total validation loss: 45.0554	Avg validation loss: 3.4658
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 346	 110	 404	 0.75877196	 0.46133333	 0.57379764
B-Disease	 204	 160	 443	 0.5604396	 0.3153014	 0.40356085
tp: 550 fp: 270 fn: 847 labels: 2
Macro-average	 prec: 0.65960574, rec: 0.38831735, f1: 0.4888457
Micro-average	 prec: 0.6707317, rec: 0.39370078, f1: 0.49616596
Quality on test dataset: 
time to finish evaluation: 1.27s
Total test loss: 52.1401	Avg test loss: 3.7243
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 357	 108	 432	 0.7677419	 0.4524715	 0.56937796
B-

As you see above, our **earlyStopping** feature worked, trainining was terminated before 30th epoch.

**Evaluating the model**

In [29]:
pred_df = ner_model.stages[2].transform(test_data)

In [30]:
pred_df.columns

['text', 'document', 'sentence', 'token', 'pos', 'label', 'embeddings', 'ner']

In [31]:
evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+-----+-----+---------+------+------+
| entity|   tp|   fp|   fn|total|precision|recall|    f1|
+-------+-----+-----+-----+-----+---------+------+------+
|Disease|584.0|166.0|120.0|704.0|   0.7787|0.8295|0.8033|
+-------+-----+-----+-----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8033012379642366|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8033012379642366|
+------------------+

None


In [32]:
evaler = medical.NerDLMetrics(mode="partial_chunk_per_token")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+------+-----+-----+------+---------+------+------+
| entity|    tp|   fp|   fn| total|precision|recall|    f1|
+-------+------+-----+-----+------+---------+------+------+
|Disease|1311.0|155.0|186.0|1497.0|   0.8943|0.8758|0.8849|
+-------+------+-----+-----+------+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8849139385757678|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8849139385757678|
+------------------+

None


`ner_utils`: This new module is used after NER training to calculate mertic chunkbase and plot training logs.

`evaluate`: if verbose, returns overall performance, as well as performance per chunk type; otherwise, simply returns overall precision, recall, f1 scores

`loss_plot`: Plots the figure of loss vs epochs

`get_charts` : Plots the figures of metrics ( precision, recall, f1) vs epochs

```
import sparknlp_jsl
from sparknlp_jsl.training_log_parser import ner_log_parser
parser = ner_log_parser()

pred_df = ner_model.stages[2].transform(test_data)

pred_df = pred_df.select(F.explode(F.arrays_zip(pred_df.ner.result,
                                                pred_df.label.result)).alias("cols"))\
                 .select(F.expr("cols['0']").alias("prediction"),
                         F.expr("cols['1']").alias("ground_truth"))
                 
df = pred_df.toPandas()


metrics = parser.evaluate( df['prediction'].values, df['ground_truth'].values)

parser.loss_plot(f"./ner_logs/{log_file}")

parser.get_charts('./ner_logs/'+log_file)
```

Saving the trained model and using in another pipeline via `load()`. 

In [33]:
ner_model.stages[2].write().overwrite().save('models/custom_NER_model')

In [36]:
document = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = nlp.SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = nlp.Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

loaded_ner_model = medical.NerModel.load("models/custom_NER_model")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

converter = medical.NerConverterInternal()\
    .setInputCols(["document", "token", "ner"])\
    .setOutputCol("ner_chunk")

ner_prediction_pipeline = nlp.Pipeline(stages=[
    document,
    sentence,
    token,
    clinical_embeddings,
    loaded_ner_model,
    converter])

empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)


light_model = nlp.LightPipeline(prediction_model)

In [37]:
text = "She has a metastatic breast cancer"

result = light_model.fullAnnotate(text)[0]

[(i.result, i.metadata['entity']) for i in result['ner_chunk']]

[('metastatic breast cancer', 'Disease')]

### Resume NER Model Training

Steps that we will follow:
- Train a new model for a few epochs.
- Load the same model and train for more epochs on the same taxnonomy, and check stats.
- Train a model already trained on a different data

**Downloading data for training**

In [38]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_test.conll
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_train_dev.conll

In [39]:
# from sparknlp.training import CoNLL

training_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_train_dev.conll')

training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
|The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [40]:
test_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_test.conll')

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Clustering of mis...|[{document, 0, 10...|[{document, 0, 10...|[{token, 0, 9, Cl...|[{pos, 0, 9, NN, ...|[{named_entity, 0...|
|Ataxia - telangie...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 5, At...|[{pos, 0, 5, NN, ...|[{named_entity, 0...|
|The risk of cance...|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



**Split the test data into two parts:**
- We Keep the first part separate and use it for training the model further, as it will be totally unseen data from the same taxonomy.

- The second part will be used to testing and evaluating

In [41]:
(test_data_1, test_data_2) = test_data.randomSplit([0.5, 0.5], seed = 100)

# save the test data as parquet for easy testing
clinical_embeddings.transform(test_data_1).write.parquet('test_1.parquet')

clinical_embeddings.transform(test_data_2).write.parquet('test_2.parquet')

#### Train a new model, pause, and resume training on the same dataset.

**Create a graph**

In [42]:
graph_folder_path = "medical_ner_graphs"

ner_graph_builder = medical.TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(24)\
    .setIsLicensed(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

**Train for 2 epochs**

In [45]:
# remove the existing logs

! rm -r ./ner_logs

In [46]:
nerTagger = medical.NerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(2)\
    .setLr(0.003)\
    .setBatchSize(8)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setTestDataset('./test_2.parquet')\
    .setGraphFolder(graph_folder_path)\
    .setOutputLogsPath('./ner_logs')

ner_pipeline = nlp.Pipeline(stages=[
    clinical_embeddings,
    ner_graph_builder,
    nerTagger
 ])

In [47]:
%%time
ner_model = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 29.1 s, sys: 249 ms, total: 29.3 s
Wall time: 4min 46s


In [48]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_3d2bcabcc184.log
ner_logs/MedicalNerApproach_a046496d4a08.log


In [49]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_20_85.pb
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 109.39s - loss: 2496.5093 - avg training loss: 3.1402633 - batches: 795
Quality on test dataset: 
time to finish evaluation: 3.59s
Total test loss: 90.5710	Avg test loss: 1.3723
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 462	 50	 121	 0.90234375	 0.7924528	 0.8438356
B-Disease	 412	 77	 106	 0.8425358	 0.7953668	 0.8182721
tp: 874 fp: 127 fn: 227 labels: 2
Macro-average	 prec: 0.87243974, rec: 0.7939098, f1: 0.83132434
Micro-average	 prec: 0.87312686, rec: 0.7938238, f1: 0.8315889


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 78.06s - loss: 1107.0641 - avg training loss: 1.3925334 - batches: 795
Quality on test dataset: 
time to finish evaluation: 1.27s
Total test loss: 66.1263	Avg test loss: 1.0019
label	 tp

**Evaluate**

In [51]:
#from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|420.0|75.0|95.0|515.0|   0.8485|0.8155|0.8317|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8316831683168316|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8316831683168316|
+------------------+

None


**Save the model to disk**

In [50]:
ner_model.stages[2].write().overwrite().save('models/NCBI_NER_2_epoch')

**Train using the saved model on unseen dataset** <br/>
We will use unseen data from the same taxonomy

In [52]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("models/NCBI_NER_2_epoch") ## load existing model
    
ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [53]:
# remove the existing logs

! rm -r ./ner_logs

In [54]:
%%time
ner_model_retrained = ner_pipeline.fit(test_data_1)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 79, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_79.pb
CPU times: user 12.1 s, sys: 72.4 ms, total: 12.1 s
Wall time: 36 s


In [55]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 434


Epoch 1/2 started, lr: 0.003, dataset size: 434


Epoch 1/2 - 7.12s - loss: 81.43485 - avg training loss: 1.4541938 - batches: 56
Quality on test dataset: 
time to finish evaluation: 2.15s
Total test loss: 64.7594	Avg test loss: 0.9812
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 535	 151	 48	 0.7798834	 0.9176672	 0.8431836
B-Disease	 439	 71	 79	 0.8607843	 0.8474904	 0.8540856
tp: 974 fp: 222 fn: 127 labels: 2
Macro-average	 prec: 0.82033384, rec: 0.8825788, f1: 0.8503188
Micro-average	 prec: 0.81438124, rec: 0.8846503, f1: 0.84806263


Epoch 2/2 started, lr: 0.0029850747, dataset size: 434


Epoch 2/2 - 5.58s - loss: 66.12025 - avg training loss: 1.1807187 - batches: 56
Quality on test dataset: 
time to finish evaluation: 1.43s
Total test loss: 69.5358	Avg test loss: 1.0536
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 465	 48	 

In [56]:
#from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label",case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+-----+
| entity|   tp|  fp|  fn|total|precision|recall|   f1|
+-------+-----+----+----+-----+---------+------+-----+
|Disease|423.0|90.0|92.0|515.0|   0.8246|0.8214|0.823|
+-------+-----+----+----+-----+---------+------+-----+

+------------------+
|             macro|
+------------------+
|0.8229571984435797|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8229571984435797|
+------------------+

None


#### Train a model already trained on a different data

We will use `ner_jsl` pretrained model for re-training. 

In [57]:
jsl_ner = medical.NerModel.pretrained('ner_jsl','en','clinical/models')

jsl_ner.getClasses()

ner_jsl download started this may take some time.
[OK!]


['O',
 'B-Injury_or_Poisoning',
 'B-Direction',
 'B-Test',
 'I-Route',
 'B-Admission_Discharge',
 'B-Death_Entity',
 'I-Oxygen_Therapy',
 'B-Relationship_Status',
 'I-Drug_BrandName',
 'B-Duration',
 'I-Alcohol',
 'I-Triglycerides',
 'I-Date',
 'B-Hyperlipidemia',
 'B-Respiration',
 'I-Test',
 'B-Birth_Entity',
 'I-VS_Finding',
 'B-Age',
 'I-Vaccine_Name',
 'I-Social_History_Header',
 'B-Labour_Delivery',
 'I-Medical_Device',
 'B-Family_History_Header',
 'B-BMI',
 'I-Fetus_NewBorn',
 'I-BMI',
 'B-Temperature',
 'I-Section_Header',
 'I-Communicable_Disease',
 'I-ImagingFindings',
 'I-Psychological_Condition',
 'I-Obesity',
 'I-Sexually_Active_or_Sexual_Orientation',
 'I-Modifier',
 'B-Alcohol',
 'I-Temperature',
 'I-Vaccine',
 'I-Symptom',
 'I-Pulse',
 'B-Kidney_Disease',
 'B-Oncological',
 'I-EKG_Findings',
 'B-Medical_History_Header',
 'I-Relationship_Status',
 'B-Cerebrovascular_Disease',
 'I-Blood_Pressure',
 'I-Diabetes',
 'B-Oxygen_Therapy',
 'B-O2_Saturation',
 'B-Psychological_C

**Now train a model using this model as base**

In [58]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("/root/cache_pretrained/ner_jsl_en_4.2.0_3.0_1666181370373")\
      .setOverrideExistingTags(True) # since the tags do not align, set this flag to true
    
# do hyperparameter by tuning the params above (max epoch, LR, dropout etc.) to get better results
ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [59]:
# remove the existing logs

! rm -r ./ner_logs

In [60]:
%%time
ner_jsl_retrained = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 13.6 s, sys: 306 ms, total: 13.9 s
Wall time: 7min 26s


In [61]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 103 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 190.58s - loss: 1321.5283 - avg training loss: 1.6622998 - batches: 795
Quality on test dataset: 
time to finish evaluation: 14.17s
Total test loss: 67.3054	Avg test loss: 1.0198
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 525	 99	 58	 0.84134614	 0.9005146	 0.86992544
B-Disease	 468	 88	 50	 0.8417266	 0.9034749	 0.8715083
tp: 993 fp: 187 fn: 108 labels: 2
Macro-average	 prec: 0.8415364, rec: 0.90199476, f1: 0.87071735
Micro-average	 prec: 0.84152544, rec: 0.9019074, f1: 0.8706708


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 187.69s - loss: 650.91437 - avg training loss: 0.8187602 - batches: 795
Quality on test dataset: 
time to finish evaluation: 11.53s
Total test loss: 48.2037	Avg test loss: 0.7304
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Dis

**Evaluating**

In [62]:
#from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_jsl_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|456.0|76.0|59.0|515.0|   0.8571|0.8854|0.8711|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8710601719197708|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8710601719197708|
+------------------+

None
